In [737]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [738]:
from deluca.agents import DRC, LQR
from deluca.envs import LDS
import jax.numpy as jnp

In [739]:
A,B = jnp.array([[.8,.5], [0,.8]]), jnp.array([[0],[0.8]])
lds = LDS(state_size= B.shape[0], action_size=B.shape[1], A=A, B=B)

In [740]:
drc = DRC(lds.A, lds.B, C=lds.C)
drc_lqr = DRC(lds.A, lds.B, C=lds.C, K= LQR(A, B).K)
lqr = LQR(A, B)

In [741]:
def get_err(T, lds, controller):
    lds.reset()
    avg_err = 0
    for i in range(T):
        action = controller(lds.obs)
        lds.step(action)
        lds.state += 0.5 * jnp.sin(i) # add sine noise
        avg_err += (jnp.linalg.norm(lds.state)+jnp.linalg.norm(action))/T
    return avg_err

In [742]:
T = 100

In [743]:
print("Pure DRC incurs ", get_err(T, lds, drc), " loss")
print("DRC initialized with LQR incurs ", get_err(T, lds, drc_lqr), " loss")
print("LQR incurs ", get_err(T, lds, lqr), " loss")

Pure DRC incurs  0.5582053  loss
DRC initialized with LQR incurs  0.7781572  loss
LQR incurs  1.3666309  loss
